<a href="https://colab.research.google.com/github/ayagup/stablediffusion/blob/main/object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
"""
Simple Object Detection Example
Quick test for object detection using Hugging Face models
"""

import torch
from transformers import AutoImageProcessor, AutoModelForObjectDetection
from PIL import Image, ImageDraw, ImageFont
import os
import gc

# Suppress warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Memory optimization
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'


def simple_object_detection(
    image_path: str,
    output_path: str = "detected_objects.jpg",
    model_name: str = "facebook/detr-resnet-50",
    confidence_threshold: float = 0.7,
):
    """
    Detect objects in an image

    Args:
        image_path: Path to input image (local file or URL)
        output_path: Path to save annotated image
        model_name: HuggingFace model name
        confidence_threshold: Minimum confidence for detections

    Returns:
        List of detected objects with bounding boxes
    """

    # Clear GPU memory from any previous runs
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()
        print("🧹 Cleared GPU memory cache\n")

    # Check GPU
    device = "cuda" if torch.cuda.is_available() else "cpu"
    num_gpus = torch.cuda.device_count() if device == "cuda" else 0

    print(f"\n{'='*60}")
    print("Simple Object Detection")
    print(f"{'='*60}")
    print(f"Device: {device}")
    if num_gpus > 0:
        print(f"GPUs: {num_gpus}")
        for i in range(num_gpus):
            print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
    print(f"{'='*60}\n")

    # Load image
    print(f"Loading image: {image_path}")
    if image_path.startswith('http://') or image_path.startswith('https://'):
        # Load from URL
        import requests
        from io import BytesIO
        response = requests.get(image_path)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    else:
        image = Image.open(image_path).convert('RGB')

    print(f"Image size: {image.size[0]}x{image.size[1]}")

    # Load model and processor
    print(f"\nLoading model: {model_name}")
    print("(This will download the model on first run)\n")

    processor = AutoImageProcessor.from_pretrained(model_name)
    model = AutoModelForObjectDetection.from_pretrained(model_name)

    # Move to GPU with memory optimization
    if device == "cuda":
        if num_gpus > 1:
            print(f"🚀 Multi-GPU mode: distributing across {num_gpus} GPUs")
            model = torch.nn.DataParallel(model)
            model = model.to(device)
        else:
            print(f"⚡ Single GPU mode")
            model = model.to(device)
    else:
        model = model.to(device)

    model.eval()

    print("Detecting objects...")

    # Prepare inputs
    inputs = processor(images=image, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Post-process
    target_sizes = torch.tensor([image.size[::-1]])  # (height, width)
    results = processor.post_process_object_detection(
        outputs,
        target_sizes=target_sizes,
        threshold=confidence_threshold
    )[0]

    # Get detections
    detections = []
    print(f"\n{'='*60}")
    print(f"Detected Objects (confidence > {confidence_threshold}):")
    print(f"{'='*60}")

    # Get model config for labels
    if hasattr(model, 'module'):
        id2label = model.module.config.id2label
    else:
        id2label = model.config.id2label

    for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
        box = [round(i, 2) for i in box.tolist()]
        label_name = id2label[label.item()]
        confidence = score.item()

        detections.append({
            "label": label_name,
            "confidence": confidence,
            "box": box
        })

        print(f"\n{label_name}")
        print(f"  Confidence: {confidence*100:.1f}%")
        print(f"  Box: [{box[0]:.0f}, {box[1]:.0f}, {box[2]:.0f}, {box[3]:.0f}]")

    print(f"\n{'='*60}")
    print(f"Total objects detected: {len(detections)}")
    print(f"{'='*60}\n")

    # Draw bounding boxes
    draw = ImageDraw.Draw(image)

    # Try to load a font, fall back to default if not available
    try:
        font = ImageFont.truetype("arial.ttf", 20)
    except:
        font = ImageFont.load_default()

    # Color palette for boxes
    colors = [
        "#FF0000", "#00FF00", "#0000FF", "#FFFF00", "#FF00FF", "#00FFFF",
        "#FFA500", "#800080", "#008000", "#FFC0CB"
    ]

    for idx, detection in enumerate(detections):
        box = detection["box"]
        label = detection["label"]
        confidence = detection["confidence"]

        # Select color
        color = colors[idx % len(colors)]

        # Draw rectangle
        draw.rectangle(box, outline=color, width=3)

        # Draw label background
        label_text = f"{label} {confidence*100:.0f}%"

        # Get text bbox (different for newer Pillow versions)
        try:
            text_bbox = draw.textbbox((box[0], box[1] - 25), label_text, font=font)
            text_width = text_bbox[2] - text_bbox[0]
            text_height = text_bbox[3] - text_bbox[1]
        except AttributeError:
            # Older Pillow version
            text_width, text_height = draw.textsize(label_text, font=font)

        # Draw background rectangle for text
        draw.rectangle(
            [box[0], box[1] - text_height - 4, box[0] + text_width + 4, box[1]],
            fill=color
        )

        # Draw text
        draw.text((box[0] + 2, box[1] - text_height - 2), label_text, fill="white", font=font)

    # Save
    image.save(output_path)

    print(f"✅ Success!")
    print(f"Annotated image saved to: {output_path}\n")

    return detections




2025-10-18 18:57:46.648197: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760813866.877944      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760813866.946849      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# if __name__ == "__main__":
# import argparse

# parser = argparse.ArgumentParser(description='Simple Object Detection')
# parser.add_argument('--image', type=str, required=True,
#                     help='Path to input image or URL')
# parser.add_argument('--output', type=str, default='detected_objects.jpg',
#                     help='Output path for annotated image')
# parser.add_argument('--model', type=str, default='facebook/detr-resnet-50',
#                     help='Model name (see README for options)')
# parser.add_argument('--threshold', type=float, default=0.7,
#                     help='Confidence threshold (0.0-1.0)')

# args = parser.parse_args()

try:
    simple_object_detection(
        image_path='https://downloadscdn5.freepik.com/download_vector/jpg/1/1308/27/27284/27284784_b4yh_ge72_220104.jpg?token=exp=1760814979~hmac=099aa84084e9df611908f65ece49077e&filename=27284784_b4yh_ge72_220104.jpg',
        output_path='/kaggle/working/detected_objects.jpg',
        model_name='facebook/detr-resnet-50',
        confidence_threshold=0.7,
    )
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()


🧹 Cleared GPU memory cache


Simple Object Detection
Device: cuda
GPUs: 1
  GPU 0: Tesla P100-PCIE-16GB

Loading image: https://downloadscdn5.freepik.com/download_vector/jpg/1/1308/27/27284/27284784_b4yh_ge72_220104.jpg?token=exp=1760814979~hmac=099aa84084e9df611908f65ece49077e&filename=27284784_b4yh_ge72_220104.jpg
Image size: 5656x4244

Loading model: facebook/detr-resnet-50
(This will download the model on first run)



Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


⚡ Single GPU mode
Detecting objects...

Detected Objects (confidence > 0.7):

scissors
  Confidence: 99.2%
  Box: [459, 2198, 1768, 3075]

Total objects detected: 1

✅ Success!
Annotated image saved to: /kaggle/working/detected_objects.jpg

